# TP2
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 1.


1. Construir uma classe Python que implemente um KEM - ElGamal. A classe deve
    1. Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits da ordem do grupo cíclico) e gere as chaves pública e privada.
    2. Conter funções para encapsulamento e revelação da chave gerada.
    3. Construir,  a partir deste KEM e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.

In [1]:
#!pip install sagemath
#!pip install pycryptodome

In [2]:
import random
import math
import secrets

from cryptography.hazmat.primitives import *
from Crypto.Util.number import getPrime
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec

### KEM - ElGamal
O KEM - ElGamal é um sistema de criptografia de chave pública que utiliza a criptografia de ElGamal como base para gerar uma chave de sessão. A chave pública consiste num par de elementos (g, h), onde g é um gerador de um grupo cíclico de ordem p e h é a imagem de g elevado a uma chave privada x.

#### 1. Inicialização
A primeira etapa é a inicialização da classe KEM_ElGamal, que receberá como entrada o parâmetro de segurança n (tamanho em bits da ordem do grupo cíclico) e gerará as chaves pública e privada.

Na inicialização da classe, o método init recebe o parâmetro de segurança n e gera os valores de p, g, x e h. O valor de p é gerado utilizando a função ``getPrime``, que retorna um número primo com n bits de tamanho. O valor de g é gerado aleatoriamente utilizando a função ``randint``, que retorna um número aleatório dentro do intervalo $[2, p-1]$. O valor de x também é gerado aleatoriamente dentro do intervalo $[2, p-2]$. Finalmente, o valor de h é calculado como a imagem de g elevado à chave privada x módulo p.

#### 2. Encapsulamento
A segunda etapa é a função de encapsulamento, que recebe a chave pública pub_key do destinatário e retorna a chave de sessão e o ciphertext.

A função ``encapsulate`` recebe a chave pública pk do destinatário, que consiste em um par de elementos (p, g, h). Em seguida, um número aleatório y é gerado dentro do intervalo $[1, p-1]$. O valor de k é calculado como a imagem de h elevado a y módulo p e o valor de s é calculado como a imagem de g elevado a y módulo p. A chave de sessão é o valor de k e o ciphertext é o par de elementos $(s, h^y)$.

#### 3. Revelação
A terceira etapa é a função `reveal`, que recebe a chave privada priv_key do destinatário e o ciphertext (s, t) e retorna a chave de sessão.

A função reveal recebe a chave privada priv_key do destinatário e o ciphertext ct (s, t), que consiste em um par de elementos $(s, h^y)$. O valor de k é calculado como a imagem de s elevado a chave privada priv_key módulo p.

In [3]:
class KEM_ElGamal:
    def __init__(self, sec_param):
        self.p = getPrime(sec_param)
        self.g = random.randint(2,self.p-1)
        self.x = random.randint(2,self.p-2)
        self.h = pow(self.g, self.x, self.p)
        self.priv_key = self.x
        self.pub_key = (self.p, self.g, self.h)
        
        """
        self.priv_key = ec.generate_private_key(ec.SECP384R1()) #self.x
        self.pub_key = self.priv_key.public_key() #(self.p, self.g, self.h)
        
        self.p = self.pub_key.public_numbers().x
        self.g = random.randint(2,self.p-1)
        self.x = random.randint(2,self.p-2)
        self.h = pow(self.g, self.x, self.p)
        """
        print(self.pub_key)

    def encapsulate(self, pub_key):
        p, g, h = pub_key
        y = random.randint(1, p-1)
        k = pow(h,y,p)
        s = pow(g,y,p)
        return k, s

    def reveal(self, priv_key, ciphertext):
        s, t = ciphertext
        k = pow(s, priv_key, self.p)
        return k


### Transformação de Fujisaki-Okamoto
A transformação de Fujisaki-Okamoto é utilizada para transformar um KEM em um PKE IND-CCA seguro. A ideia é que a chave de sessão gerada pelo KEM seja utilizada como chave para criptografar a mensagem utilizando uma cifra simétrica.

#### 1. Inicialização
A primeira etapa é a inicialização da classe `PKE_F`, que recebe como entrada o parâmetro de segurança n (tamanho em bits da ordem do grupo cíclico) e um objeto da classe`KEM_ElGamal`.

Na inicialização da classe, o método `ini` recebe o parâmetro de segurança n e um objeto da classe KEM_ElGamal. O atributo kem armazena o objeto da classe KEM e o atributo key_size armazena o tamanho da chave da cifra simétrica em bytes.

#### 2. Cifragem
A segunda etapa é a função de cifrar, que recebe a chave pública pub_key do destinatário e a mensagem msg e retorna o ciphertext.

A função encrypt recebe a chave pública pub_key do destinatário e a mensagem msg. Em seguida, a chave de sessão k e o valor s são gerados utilizando a função `encapsulat` do objeto kem da classe`KEM_ElGamal`. A chave de sessão k é convertida para um objeto bytes utilizando o método `to_byte` e o tamanho da chave da cifra simétrica em bytes. A cifra simétrica **AES** é utilizada para criptografar a mensagem msg utilizando a chave de sessão k. O método `pad` é utilizado para adicionar *padding* à mensagem para que ela tenha um tamanho múltiplo do tamanho da chave da cifra simétrica. O ciphertext é o par de elementos $(s, ct, iv)$, onde ct é a mensagem cifrada, s é um valor utilizado na decifragem e iv é o vetor de inicialização utilizado na cifra simétrica.

#### 3. Decifragem
A terceira etapa é a função de decifrar, que recebe a chave privada priv_key do destinatário e o ciphertext c e retorna a mensagem original.

Na função decrypt, a chave privada priv_key do destinatário e o ciphertext c são recebidos como entrada. O valor s, a mensagem cifrada ct e o vetor de inicialização iv são extraídos do ciphertext c. A chave de sessão k é recuperada utilizando a função `reveal` do objeto kem da classe `KEM_ElGamal`. O valor s e o vetor de inicialização iv são utilizados como entrada para a função `reveal`, pois são usados para calcular a chave de sessão k durante o encapsulamento. A chave de sessão k é convertida para um objeto bytes utilizando o método `to_bytes` e o tamanho da chave da cifra simétrica em bytes. A cifra simétrica AES é utilizada para decifrar a mensagem cifrada ct usando a chave de sessão k e o vetor de inicialização iv. O método `unpad` é utilizado para remover o padding da mensagem decifrada e retornar a mensagem original.

In [4]:
"""
class PKE_FujisakiOkamoto:
    def __init__(self, sec_param, kem):
        self.sec_param = sec_param
        self.kem = kem
        self.key_size = 16

    def encrypt(self, pub_key, msg):
        
        k, s = self.kem.encapsulate(pub_key)
        
        aes_key = k.to_bytes(self.key_size, byteorder='big')
        cipher = AES.new(aes_key, AES.MODE_CBC)
        padded_msg = pad(msg, self.key_size)
        ciphertext = cipher.encrypt(padded_msg)
        c = (s, ciphertext, cipher.iv)
                    
        return c

    def decrypt(self, priv_key, ciphertext):
        s, ct, iv = ciphertext
        k = self.kem.reveal(priv_key, (s,iv))
        aes_key = k.to_bytes(self.key_size, byteorder='big')
        cipher = AES.new(aes_key, AES.MODE_CBC, iv)
        msg = cipher.decrypt(ct)
        unpadded_msg = unpad(msg, self.key_size)
        
        return unpadded_msg
    """

class PKE_FujisakiOkamoto:
    def __init__(self, n, kem):
        self.n = n
        self.kem = kem
        self.key_size = 16

    def encrypt(self, pk, msg):
        k, s = self.kem.encapsulate(pk)
        aes_key = k.to_bytes(self.key_size, byteorder='big')
        iv = secrets.token_bytes(self.key_size)  # gera vetor de inicialização aleatório
        cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
        padder = padding.PKCS7(self.key_size * 8).padder()
        padded_msg = padder.update(msg) + padder.finalize()
        encryptor = cipher.encryptor()
        ct = encryptor.update(padded_msg) + encryptor.finalize()
        c = (s, ct, iv)
        return c

    def decrypt(self, priv_key, ct):
        s, ct, iv = ct
        k = self.kem.reveal(priv_key, (s, iv))
        aes_key = k.to_bytes(self.key_size, byteorder='big')
        cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
        decryptor = cipher.decryptor()
        unpadded_msg = decryptor.update(ct) + decryptor.finalize()
        unpadder = padding.PKCS7(self.key_size * 8).unpadder()
        msg = unpadder.update(unpadded_msg) + unpadder.finalize()
        return msg


In [5]:
"""
msg = b"estruturas criptograficas"

sec_param = 128

kem = KEM_ElGamal(sec_param)
pke = PKE_FujisakiOkamoto(128, kem)

ciphertext = pke.encrypt(kem.pub_key, msg)
plaintext = pke.decrypt(kem.priv_key, ciphertext)


print("Plaintext: " + plaintext.decode())
"""

# instantiate a KEM object with a security parameter of 2048 bits
kem = KEM_ElGamal(random.randint(0, 128))

# instantiate a PKE object using the KEM object
pke = PKE_FujisakiOkamoto(16, kem)

# generate a public key and a private key
pub_key = kem.pub_key
priv_key = kem.priv_key

# message to be encrypted
msg = b"Estruturas Criptograficas - Grupo 17!"

# encrypt the message using the public key
ciphertext = pke.encrypt(pub_key, msg)

# decrypt the ciphertext using the private key
decrypted_msg = pke.decrypt(priv_key, ciphertext)

# print the original message and the decrypted message to verify that they match
print("Mensagem Original: ", msg)
# print("Mensagem Cifrada: ", ciphertext)
print("Mensagem Decifrada: ", decrypted_msg)

(117973, 21184, 18224)
Mensagem Original:  b'Estruturas Criptograficas - Grupo 17!'
Mensagem Decifrada:  b'Estruturas Criptograficas - Grupo 17!'
